<a href="https://colab.research.google.com/github/harnalashok/LLMs/blob/main/chromadbExpts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 31st May, 2024
# Objective: a. IF possible, transform chromadb dataset to csv
#            b. Export chromadb datasets to huggingface

## References:
> a. [Huggingface](https://huggingface.co/chromadb)    
> b. [GitHub-1](https://github.com/chroma-core/chroma_datasets/tree/master)    
> c. [GitHub-2](https://github.com/johnnycode8/chromadb_quickstart/tree/main)      
> d. [Analytics Vidya](https://www.analyticsvidhya.com/blog/2023/07/guide-to-chroma-db-a-vector-store-for-your-generative-ai-llms/)    
> e. [chromadb cookbook](https://cookbook.chromadb.dev/)

In [1]:
# 1.0 chroma db is installed as a python package
#       not as an ubuntu software:

! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5

## Why chromadb?
Here are the steps describing how Chroma DB works:

- **Data Structure**: Chroma DB organizes chromatic data in a structured format optimized for efficient storage and retrieval.
- **Storage**: It stores metadat also. For images, it stores color-related information such as RGB values, color names, and associated metadata in the database.
- **Indexing**: Metadata is indexed. Chroma DB creates indexes to facilitate fast lookup of colors based on various criteria like RGB values, color names, or other attributes.
- **Querying**: Query can include metadata. Users can query Chroma DB using specific criteria such as color codes, names, or properties to retrieve relevant color information.
- **Analysis**: Chroma DB enables analysis of color data for various applications such as image processing, design, and color matching.
- **Optimization**: The database is optimized for speed and efficiency, allowing for quick retrieval and processing of color-related information.
- **Integration**: It can be integrated into different software applications and platforms to provide color-related functionalities seamlessly.
- **Continued Improvement**: Chroma DB may undergo updates and improvements to enhance its capabilities and accommodate evolving requirements in color management and analysis.


### Structure

- Tenants: A tenant is a logical grouping of databases. Default is `DEFAULT_TENANT`
- Databases: A database is a logical grouping of collections (tables). Default is `DEFAULT_DATABASE`
- A Collection is akin toa table in RDBMS

In [2]:
# 1.0
# chroma cookbook: https://cookbook.chromadb.dev/core/tenants-and-databases/
# chroma db has Tenants: A tenant is a logical grouping of databases.
# chromadb has Databases: A database is a logical grouping of collections (tables).

import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

## Clients

chroma has tow types of clients unlike in databases. One client (PersistentClient) is suited only for local chroma server and the other client (HttpClient) for remote chroma server.

In [3]:
# 1.1 Create a localclient:
# https://cookbook.chromadb.dev/core/clients/#persistent-client

# 1.1.1 No host is specified. Understood to be the same where this code runs:
myclient = chromadb.PersistentClient(
                                      path="/content/",      # Where data will be stored
                                      settings=Settings(),   # Default settungs
                                      tenant=DEFAULT_TENANT, # Default
                                      database=DEFAULT_DATABASE,
                                     )


In [ ]:
# 1.1.2  YOu can try this also. But host has to be specified.
#        But in colab, it will not work as in colab, there is no localhost:
#        Ref: https://cookbook.chromadb.dev/core/clients/#http-client

client = chromadb.HttpClient(
                              host="localhost",
                              port=8000,
                              ssl=False,
                              headers=None,
                              settings=Settings(),
                              tenant=DEFAULT_TENANT,
                              database=DEFAULT_DATABASE,
                          )

## Collections

Collection in Vector Store is where we save the set of a) vector embeddings, b) documents, and c) any metadata, if present. Collection in a vector database can be thought of as a Table in Relational Database.

### [Collection Properties](https://cookbook.chromadb.dev/core/collections/#collection-properties)¶

Each collection is characterized by the following properties:

- name: The name of the collection. The name can be changed as long as it is unique within the database ( use collection.modify(new_name="new_name") to change the name of the collection
- metadata: A dictionary of metadata associated with the collection. The metadata is a dictionary of key-value pairs. Keys can be strings, values can be strings, integers, floats, or booleans. Metadata can be changed using collection.modify(new_metadata={"key": "value"}) (Note: Metadata is always overwritten when modified)
- embedding_function: The embedding function used to embed documents in the collection.

Defaults:

- Embedding Function - by default if `embedding_function` parameter is not provided at `get()` or `create_collection()` or `get_or_create_collection()` time, Chroma uses `chromadb.utils.embedding_functions`.`DefaultEmbeddingFunction` which uses the chromadb.`utils.embedding_functions`.`DefaultEmbeddingFunction` to embed documents. The default embedding function uses `Onnx Runtime` with `all-MiniLM-L6-v2` model. **Sentence transformer** model will be downloaded when we add documents to collection. Refer [here](https://docs.trychroma.com/guides/embeddings#default:-all-minilm-l6-v2) for documentation on *Embeddings*.        

-distance metric - by default Chroma use L2 (Euclidean Distance Squared) distance metric for newly created collection. You can change it at creation time using hnsw:space metadata key. Possible values are l2, cosine, and 'ip' (inner product)

### Embedding functions
Embedding functions can be linked to a collection and used whenever you call `add`, `update`, `upsert` or `query`. You can also use them directly which can be handy for debugging. Here is an example of using them directly:

By default, Chroma uses the **Sentence Transformers** `all-MiniLM-L6-v2` model to create embeddings. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks. This embedding function runs locally on your machine, and may require you download the model files (this will happen automatically).

In [ ]:
# 2.0
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()
val = default_ef(["foo"])
val

Chroma can also use any **Sentence Transformers** model to create embeddings.    
List of all Sentence Transformer models is [here](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html)

In [12]:
# 2.1 You have to first install sentence_transformers package:
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [13]:
# 2.2 The model will first be downloaded and then used:

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Create collections
And add to them

In [4]:
# 3.0 We will now create an empty collection and add documents to it.
#     Sentence transformer model will be downloaded when we add documents.

mycollection = myclient.create_collection("my_information")

To this collection, we will be adding documents. Here we are adding three documents. In our case, we are just adding three sentences as three documents. The first document is about cars, the second one is about dogs and the final one is about four-wheelers.    
We are even adding the metadata. Metadata for all three documents is provided.     
Every document needs to have a unique ID to it, hence we are giving id1, id2, and id3 to them.     
All these are like the variables to the add() function from the collection

In [5]:
# 3.1 Note the model that is downloaded (all-MiniLM-L6-v2/onnx.tar.gz)
#        and also note the place where it is downloaded:

mycollection.add(
                documents=["This is a document containing car information",
                "This is a document containing information about dogs",
                "This document contains four wheeler catalogue"],
                metadatas=[{"source": "Car Book"},{"source": "Dog Book"},{'source':'Vechile Info'}],
                ids=["id1", "id2", "id3"]
               )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 35.6MiB/s]


## Query a Vector Store

We will pass a query and will fetch the documents that are relevant to it. The corresponding code for this will be     

- To query a vector store, we have a query() function provided by the collections which lets us query the vector database for relevant documents. In this function, we provide two parameters
- query_texts – To this parameter, we give a list of queries for which we need to extract the relevant documents.
- n_results - This parameter specifies how many top results should the database return. In our case we want our collection to return 2 top most relevant documents related to the query

When we gave a query, the Chrom DB converts the query into a vector embedding with the embedding model we provided at the start. Then this vector embedding performs a semantic search(similar nearest neighbors) on all the available documents. The query here “car” is most relevant to the id1 and id3 documents, hence we get the following result for the query.

In [ ]:
# 4.0
results = mycollection.query(
                            query_texts=["Car"],
                            n_results=2
                            )


# 4.1
print(results)

{'ids': [['id1', 'id3']], 'distances': [[1.0784918423508782, 1.5855050680528366]], 'metadatas': [[{'source': 'Car Book'}, {'source': 'Vechile Info'}]], 'embeddings': None, 'documents': [['This is a document containing car information', 'This document contains four wheeler catalogue']], 'uris': None, 'data': None}


## Update

Later in point of time, as and when we get more data then it becomes necessary to update the existing data/vector embeddings present in the Vector Store. To update data in Chroma DB, we do the following

Previously, the information in the document associated with id2 was about Dogs. Now we are changing it to Cats. For this information to be updated within the Vector Store, we pass the id of the document, the updated document, and the updated metadata of the document to the update() function of the collections. This will now update the id2 to Cats which was previously about Dogs.

In [ ]:
# 5.0
mycollection.update(
                    ids=["id2"],
                    documents=["This is a document containing information about Cats"],
                    metadatas=[{"source": "Cat Book"}],
                  )

In [ ]:
# 5.1 Query the database again:

results = mycollection.query(
                              query_texts=["Felines"],
                              n_results=1
                            )

# 5.2
print(results)

{'ids': [['id2']], 'distances': [[0.9560528333913512]], 'metadatas': [[{'source': 'Cat Book'}]], 'embeddings': None, 'documents': [['This is a document containing information about Cats']], 'uris': None, 'data': None}


Sometimes, to reduce the space or remove unnecessary/ unwanted information, we might want to delete some documents from the collection in the Vector Store.

## Delete

In [ ]:
# 6.0
mycollection.delete(ids = ['id1'])

# 6.1
results = mycollection.query(
                            query_texts=["Car"],
                            n_results=2
                            )

# 6.2
print(results)

{'ids': [['id3', 'id2']], 'distances': [[1.5855050680528366, 1.7295208916874063]], 'metadatas': [[{'source': 'Vechile Info'}, {'source': 'Cat Book'}]], 'embeddings': None, 'documents': [['This document contains four wheeler catalogue', 'This is a document containing information about Cats']], 'uris': None, 'data': None}


`count()` an `get()` functions

## Misc

In [ ]:
# 7.0
new_collections = myclient.create_collection("new_collection")

# 7.1
new_collections.add(
                    documents=["This is Python Documentation",
                              "This is a Javascript Documentation",
                              "This document contains Flast API Cheatsheet"],
                    metadatas=[{"source": "Python For Everyone"},
                    {"source": "JS Docs"},
                    {'source':'Everything Flask'}],
                    ids=["id1", "id2", "id3"]
                  )


# 7.2
print(new_collections.count())
print(new_collections.get())

3
{'ids': ['id1', 'id2', 'id3'], 'embeddings': None, 'metadatas': [{'source': 'Python For Everyone'}, {'source': 'JS Docs'}, {'source': 'Everything Flask'}], 'documents': ['This is Python Documentation', 'This is a Javascript Documentation', 'This document contains Flast API Cheatsheet'], 'uris': None, 'data': None}


In [ ]:
########## DONE #######